<a href="https://colab.research.google.com/github/smitha-google/DataPipelineFromAzure/blob/main/DataPipelineFromAzure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set Variables**




In [3]:
# Set the Project id
project_id = 'smithaargolisinternal'

#Connect to MimeoDataset GCS bucket
bucket_name="mimeodata"

#PubSub Topic Name
topic_name="mimeotopic" 

#PubSub Subscription Name
subscription_name="mimeosubscription"

#BigQuery Dataset Id
dataset_id="mimeo_dataset"

#BigQuery Table Id
table_id="yellowtable"

#GCS Bucket Name to store Functions Data
functions_bucket="smithaargolisinternal-functions"

#Region
region="us-central1"

#AzureSAS Key
sas_key="ABC"

**Authenticate to Access GCS**

In [5]:
# Authenticate to Access GCS
from google.colab import auth
auth.authenticate_user()

# gsutil to use the project we specified by using gcloud
!gcloud config set project {project_id}

Updated property [core/project].


**Testing Access**

In [6]:
#List the files in the GCS Bucket
!gsutil ls gs://{bucket_name}/

gs://mimeodata/bquxjob_5fb8f53c_1807602e66b.json
gs://mimeodata/mimeo_data.csv
gs://mimeodata/test-finalize.txt
gs://mimeodata/ADLSData/


**Storage Transfer Service From Azure**

In [ ]:
!gcloud transfer jobs create \
  https://smtestaccount.blob.core.windows.net/smtestcontainer gs://{bucket_name}\
  --source-creds-file={sas_key} \
  --include-modified-after-relative=1H \
  --schedule-repeats-every=1H`

**Cloud Functions to Load to BQ**

Code is in Github-function-source.zip. Deploy the code and ensure the right service account is used to invoke this function. 

**Check in BigQuery**

In [7]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

In [8]:
# print 10 rows
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,653570,99999,1977,12,2,82.199997,8,39.599998,8.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
1,839370,99999,1993,9,1,57.900002,5,57.900002,5.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
2,114570,99999,1997,6,24,46.799999,8,41.400002,8.0,NaN,...,None,None,0.08,NaN,False,False,False,False,False,False
3,333170,99999,1952,7,22,62.000000,4,NaN,NaN,1019.000000,...,None,None,0.00,NaN,False,False,False,False,False,False
4,939860,99999,1981,11,3,48.200001,4,43.200001,4.0,1003.599976,...,None,None,NaN,NaN,False,False,False,False,False,False
5,670720,99999,2001,11,7,79.500000,4,72.000000,4.0,1018.599976,...,None,None,0.12,NaN,False,False,False,False,False,False
6,80800,99999,1995,7,30,69.400002,18,58.799999,18.0,1014.500000,...,None,None,0.00,NaN,False,False,False,False,False,False
7,871780,99999,1986,9,12,73.900002,6,56.200001,6.0,1012.299988,...,None,None,0.00,NaN,False,False,False,False,False,False
8,409900,99999,1977,2,15,49.500000,7,26.900000,7.0,1024.400024,...,None,None,0.00,NaN,True,True,True,True,True,True
9,66700,99999,1963,9,11,56.099998,8,51.299999,8.0,1013.799988,...,None,None,0.01,NaN,True,True,True,True,True,True
